# Reportagem Seriada: Censo do Ensino Superior

Neste notebook, analisamos os microdados do Censo da Educação Superior, realizado anualmente pelo Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira (Inep) para entender como tem se comportado o cenário do financiamento estudantil para matrículas em Instituiçõs de Ensino Superior (IES) no Ceará.

**ÍNDICE**

1. [Coleta, Limpeza e Tratamento de Dados](#coleta)

    1.1 [Extrair informações de localização](#extrair)
    
    1.2 [Dados das instituições](#dados-instituicoes)
    
    1.3 [Dados dos cursos e alunos](#dados-cursos-alunos)
    
    1.4 [Dataset total](#dataset-total)
    
    
2. [Análises e recortes para visualizações](#analises)

    2.1 [Proporção e números absolutos de matrículas com financiamento em relação ao total de alunos matriculados na rede privada de ensino superior no Ceará](#proporcao-e-absolutos)
     
    2.2 [Números específicos de ingressantes e concluintes com financiamento estudantil](#ingressantes-e-concluintes)
    
    2.3 [Proporção e números absolutos de matrículas na rede privada com financiamento estudantil em Fortaleza e no Interior do estado](#matriculas-com-financiamento)
    
    2.4 [Total de alunos com e sem financiamento estudantil da rede privada de ensino superior do Ceará](#total-com-e-sem-financiamento)
    
    2.5 [Grandes áreas e tipos de grau acadêmico o financiamento estudantil no Ceará](#grandes-areas)
    
    2.6 [Outras consultas (Extra)](#extra)

## 1. Coleta, Limpeza e Tratamento de Dados <a name="coleta"></a>

In [ ]:
!pip install geopandas

In [1]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries
import shapely
from shapely.geometry import Polygon
from shapely.geometry import Point

In [2]:
pd.set_option("display.max_columns", 90)

In [3]:
graficos = 'graficos/'

In [4]:
limites_ce = 'Limites_municipais_Ceara2021/'

### 1.1 Extrair informações de localização <a name="extrair"></a>

In [5]:
shapeFileCE = gpd.read_file(limites_ce + 'Limites_municipais_IPECE_2021_utm_sirgas_2000.shp')

In [6]:
mapa_ceara = pd.read_csv('mapaDoCeara.csv')

In [7]:
mapa_ceara['CO_MUNICIPIO'] = mapa_ceara['CO_MUNICIPIO'].astype(str)
mapa_ceara['CO_MUNICIPIO_6'] = mapa_ceara['CO_MUNICIPIO_6'].astype(str)

In [8]:
arquivo_mapa_ceara = shapeFileCE[['Municipio', 'regiao', 'codigo_ibg']].merge(mapa_ceara, left_on='codigo_ibg', right_on='CO_MUNICIPIO')
arquivo_mapa_ceara = arquivo_mapa_ceara[['GEOMETRY', 'CO_MUNICIPIO', 'CO_MUNICIPIO_6', 'Municipio', 'regiao']]

### 1.2 Dados das instituições <a name="dados-instituicoes"></a>

In [9]:
anos = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
lista_instituicao = []

for ano in anos: 
    instituicao = pd.read_csv(f'Microdados do Censo da Educação Superior {ano}/dados/MICRODADOS_CADASTRO_IES_{ano}.CSV', encoding='latin-1', sep=';')
    lista_instituicao.append(instituicao)

df_instituicoes = pd.concat(lista_instituicao)

In [10]:
inst_ce = df_instituicoes[df_instituicoes['SG_UF_IES'] == 'CE']

In [11]:
df_inst_ce = inst_ce[['NU_ANO_CENSO', 'SG_UF_IES', 'CO_MUNICIPIO_IES', 'NO_MUNICIPIO_IES', 'IN_CAPITAL_IES',
                      'NO_MICRORREGIAO_IES', 'TP_ORGANIZACAO_ACADEMICA', 'TP_CATEGORIA_ADMINISTRATIVA', 'CO_IES', 'NO_IES', 'SG_IES']]

In [12]:
# TP_ORGANIZACAO_ACADEMICA

organizacao_academica = {
    1: 'Universidade',
    2: 'Centro Universitário',
    3: 'Faculdade',
    4: 'Instituto Federal de Educação, Ciência e Tecnologia',
    5: 'Centro Federal de Educação Tecnológica'
}

df_inst_ce['TP_ORGANIZACAO_ACADEMICA'] = df_inst_ce['TP_ORGANIZACAO_ACADEMICA'].replace(organizacao_academica)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\947665288.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inst_ce['TP_ORGANIZACAO_ACADEMICA'] = df_inst_ce['TP_ORGANIZACAO_ACADEMICA'].replace(organizacao_academica)


In [13]:
# IN_CAPITAL_IES

capital = {
    1: 'Capital',
    0: 'Interior'
}
df_inst_ce['IN_CAPITAL_IES'] = df_inst_ce['IN_CAPITAL_IES'].replace(capital)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\3282651258.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inst_ce['IN_CAPITAL_IES'] = df_inst_ce['IN_CAPITAL_IES'].replace(capital)


In [14]:
# TP_CATEGORIA_ADMINISTRATIVA

categoria_administrativa = {
    1: 'Pública Federal',
    2: 'Pública Estadual',
    3: 'Pública Municipal',
    4: 'Privada com fins lucrativos',
    5: 'Privada sem fins lucrativos',
    6: 'Privada - Particular em sentido estrito',
    7: 'Especial',
    8: 'Privada comunitária',
    9: 'Privada confessional'
}

df_inst_ce['TP_CATEGORIA_ADMINISTRATIVA'] = df_inst_ce['TP_CATEGORIA_ADMINISTRATIVA'].replace(categoria_administrativa)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\747519099.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inst_ce['TP_CATEGORIA_ADMINISTRATIVA'] = df_inst_ce['TP_CATEGORIA_ADMINISTRATIVA'].replace(categoria_administrativa)


In [15]:
df_inst_ce['CO_IES'] = df_inst_ce['CO_IES'].map('{:06d}'.format)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\552071556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inst_ce['CO_IES'] = df_inst_ce['CO_IES'].map('{:06d}'.format)


In [16]:
def strTransform(df, col):
    return df[col].astype(str)

In [17]:
df_inst_ce['CO_MUNICIPIO_IES'] = strTransform(df_inst_ce, 'CO_MUNICIPIO_IES')
df_inst_ce['NU_ANO_CENSO'] = strTransform(df_inst_ce, 'NU_ANO_CENSO')

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\4083808529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inst_ce['CO_MUNICIPIO_IES'] = strTransform(df_inst_ce, 'CO_MUNICIPIO_IES')
C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\4083808529.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inst_ce['NU_ANO_CENSO'] = strTransform(df_inst_ce, 'NU_ANO_CENSO')


In [18]:
df_inst_ce.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 772 entries, 26 to 2562
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   NU_ANO_CENSO                 772 non-null    object
 1   SG_UF_IES                    772 non-null    object
 2   CO_MUNICIPIO_IES             772 non-null    object
 3   NO_MUNICIPIO_IES             772 non-null    object
 4   IN_CAPITAL_IES               772 non-null    object
 5   NO_MICRORREGIAO_IES          772 non-null    object
 6   TP_ORGANIZACAO_ACADEMICA     772 non-null    object
 7   TP_CATEGORIA_ADMINISTRATIVA  772 non-null    object
 8   CO_IES                       772 non-null    object
 9   NO_IES                       772 non-null    object
 10  SG_IES                       686 non-null    object
dtypes: object(11)
memory usage: 72.4+ KB


### 1.3 Dados dos cursos e alunos <a name="dados-cursos-alunos"></a>

In [19]:
# importação da base filtrada

cursos_ce = pd.read_csv('df_cursos_ce.csv')

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\954320499.py:3: DtypeWarning: Columns (17,200) have mixed types. Specify dtype option on import or set low_memory=False.
  cursos_ce = pd.read_csv('df_cursos_ce.csv')


In [20]:
# QT_ING: Cálculo de ingressantes: soma do número de alunos com data de ingresso de 01 de janeiro e 01 de julho do ano de referência do censo
# QT_MAT: Cálculo de matrículas: soma do número de alunos com situação de vínculo ao curso igual a: Cursando e/ou Formado

# QT_ING_FINANC: Quantidade de ingressantes - alunos que utilizam financiamento estudantil 
# QT_ING_FINANC_REEMB: Quantidade de ingressantes - alunos que utilizam financiamento estudantil reembolsável
# QT_ING_FIES: Quantidade de ingressantes - alunos que utilizam financiamento estudantil reembolsável  - FIES
# QT_ING_RPFIES: Quantidade de ingressantes - alunos que utilizam financiamento estudantil reembolsável - administrado pela instituição
# QT_ING_FINANC_REEMB_OUTROS: Quantidade de ingressantes - alunos que utilizam financiamento estudantil reembolsável - pelo governo estadual, municipal, entidades externas à IES e outras entidades
# QT_ING_FINANC_NREEMB: Quantidade de ingressantes - alunos que utilizam financiamento estudantil não reembolsável
# QT_ING_PROUNII: Quantidade de ingressantes - alunos que utilizam financiamento estudantil não reembolsável  - PROUNI Integral
# QT_ING_PROUNIP: Quantidade de ingressantes - alunos que utilizam financiamento estudantil não reembolsável  - PROUNI Parcial
# QT_ING_NRPFIES: Quantidade de ingressantes - alunos que utilizam financiamento estudantil não reembolsável - administrado pela instituição
# QT_ING_FINANC_NREEMB_OUTROS: Quantidade de ingressantes - alunos que utilizam financiamento estudantil não reembolsável - pelo governo estadual, municipal, entidades externas à IES e outras entidades

# QT_MAT_FINANC: Quantidade de matrículas - alunos que utilizam financiamento estudantil 
# QT_MAT_FINANC_REEMB: Quantidade de matrículas - alunos que utilizam financiamento estudantil  reembolsável
# QT_MAT_FIES: Quantidade de matrículas - alunos que utilizam financiamento estudantil reembolsável  - FIES
# QT_MAT_RPFIES: Quantidade de matrículas - alunos que utilizam financiamento estudantil reembolsável - administrado pela instituição
# QT_MAT_FINANC_REEMB_OUTROS: Quantidade de matrículas - alunos que utilizam financiamento estudantil reembolsável - pelo governo estadual, municipal, entidades externas à IES e outras entidades
# QT_MAT_FINANC_NREEMB: Quantidade de matrículas - alunos que utilizam financiamento estudantil não reembolsável
# QT_MAT_PROUNII: Quantidade de matrículas - alunos que utilizam financiamento estudantil não reembolsável  - PROUNI Integral
# QT_MAT_PROUNIP: Quantidade de matrículas - alunos que utilizam financiamento estudantil não reembolsável  - PROUNI Parcial
# QT_MAT_NRPFIES: Quantidade de matrículas - alunos que utilizam financiamento estudantil não reembolsável - administrado pela instituição
# QT_MAT_FINANC_NREEMB_OUTROS: Quantidade de matrículas - alunos que utilizam financiamento estudantil não reembolsável - pelo governo estadual, municipal, entidades externas à IES e outras entidades

# QT_CONC_FINANC: Quantidade de concluintes - alunos que utilizam financiamento estudantil 
# QT_CONC_FINANC_REEMB: Quantidade de concluintes - alunos que utilizam financiamento estudantil reembolsável
# QT_CONC_FIES: Quantidade de concluintes - alunos que utilizam financiamento estudantil reembolsável  - FIES
# QT_CONC_RPFIES: Quantidade de concluintes - alunos que utilizam financiamento estudantil reembolsável - administrado pela instituição
# QT_CONC_FINANC_REEMB_OUTROS: Quantidade de concluintes - alunos que utilizam financiamento estudantil reembolsável - pelo governo estadual, municipal, entidades externas à IES e outras entidades
# QT_CONC_FINANC_NREEMB: Quantidade de concluintes - alunos que utilizam financiamento estudantil não reembolsável
# QT_CONC_PROUNII: Quantidade de concluintes - alunos que utilizam financiamento estudantil não reembolsável  - PROUNI Integral
# QT_CONC_PROUNIP: Quantidade de concluintes - alunos que utilizam financiamento estudantil não reembolsável  - PROUNI Parcial
# QT_CONC_NRPFIES: Quantidade de concluintes - alunos que utilizam financiamento estudantil não reembolsável - administrado pela instituição
# QT_CONC_FINANC_NREEMB_OUTROS: Quantidade de concluintes - alunos que utilizam financiamento estudantil não reembolsável - pelo governo estadual, municipal, entidades externas à IES e outras entidades

In [21]:
df_cursos_ce = cursos_ce[['NU_ANO_CENSO', 'NO_MUNICIPIO', 'CO_MUNICIPIO', 'IN_CAPITAL', 'CO_IES', 'TP_REDE', 'NO_CURSO', 'NO_CINE_ROTULO', 'NO_CINE_AREA_GERAL', 'TP_GRAU_ACADEMICO', 'QT_CURSO',
                          'QT_ING',
                          'QT_ING_FINANC', 'QT_ING_FINANC_REEMB', 'QT_ING_FIES', 'QT_ING_RPFIES', 'QT_ING_FINANC_REEMB_OUTROS', 'QT_ING_FINANC_NREEMB', 'QT_ING_PROUNII', 'QT_ING_PROUNIP', 'QT_ING_NRPFIES', 'QT_ING_FINANC_NREEMB_OUTROS',
                          'QT_MAT',
                          'QT_MAT_0_17', 'QT_MAT_18_24', 'QT_MAT_25_29', 'QT_MAT_30_34',  'QT_MAT_35_39', 'QT_MAT_40_49', 'QT_MAT_50_59', 'QT_MAT_60_MAIS',
                          'QT_MAT_FINANC', 'QT_MAT_FINANC_REEMB', 'QT_MAT_FIES', 'QT_MAT_RPFIES', 'QT_MAT_FINANC_REEMB_OUTROS', 'QT_MAT_FINANC_NREEMB', 'QT_MAT_PROUNII', 'QT_MAT_PROUNIP', 'QT_MAT_NRPFIES', 'QT_MAT_FINANC_NREEMB_OUTROS',
                          'QT_CONC',
                          'QT_CONC_FINANC', 'QT_CONC_FINANC_REEMB', 'QT_CONC_FIES', 'QT_CONC_RPFIES', 'QT_CONC_FINANC_REEMB_OUTROS', 'QT_CONC_FINANC_NREEMB', 'QT_CONC_PROUNII', 'QT_CONC_PROUNIP', 'QT_CONC_NRPFIES', 'QT_CONC_FINANC_NREEMB_OUTROS','QT_VG_TOTAL']]

In [22]:
def floatToInt(df, col):
    return df[col].astype(int)

In [23]:
df_cursos_ce['TP_GRAU_ACADEMICO'].fillna(0, inplace=True)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\1367729694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cursos_ce['TP_GRAU_ACADEMICO'].fillna(0, inplace=True)


In [24]:
df_cursos_ce['IN_CAPITAL'] = floatToInt(df_cursos_ce, 'IN_CAPITAL')
df_cursos_ce['CO_MUNICIPIO'] = floatToInt(df_cursos_ce, 'CO_MUNICIPIO')
df_cursos_ce['TP_GRAU_ACADEMICO'] = floatToInt(df_cursos_ce, 'TP_GRAU_ACADEMICO')
df_cursos_ce['CO_MUNICIPIO'] = strTransform(df_cursos_ce, 'CO_MUNICIPIO')
df_cursos_ce['NU_ANO_CENSO'] = strTransform(df_cursos_ce, 'NU_ANO_CENSO')

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\886602562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cursos_ce['IN_CAPITAL'] = floatToInt(df_cursos_ce, 'IN_CAPITAL')
C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\886602562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cursos_ce['CO_MUNICIPIO'] = floatToInt(df_cursos_ce, 'CO_MUNICIPIO')
C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\886602562.py:3: SettingWithCopyWarning: 
A value is tryi

In [25]:
# TP_GRAU_ACADEMICO

grau_academico = {
    1: 'Bacharelado',
    2: 'Licenciatura',
    3: 'Tecnológico',
    4: 'Bacharelado e Licenciatura',
    0: 'Não aplicável'
}

df_cursos_ce['TP_GRAU_ACADEMICO'] = df_cursos_ce['TP_GRAU_ACADEMICO'].replace(grau_academico)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\4250632500.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cursos_ce['TP_GRAU_ACADEMICO'] = df_cursos_ce['TP_GRAU_ACADEMICO'].replace(grau_academico)


In [26]:
df_cursos_ce['IN_CAPITAL'] = df_cursos_ce['IN_CAPITAL'].replace(capital)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\2801993769.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cursos_ce['IN_CAPITAL'] = df_cursos_ce['IN_CAPITAL'].replace(capital)


In [27]:
df_cursos_ce['CO_IES'] = df_cursos_ce['CO_IES'].map('{:06d}'.format)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\3233285610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cursos_ce['CO_IES'] = df_cursos_ce['CO_IES'].map('{:06d}'.format)


In [28]:
# TP_REDE

tp_rede = {
    1: 'Pública',
    2: 'Privada'
}

df_cursos_ce['TP_REDE'] = df_cursos_ce['TP_REDE'].replace(tp_rede)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\1934030802.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cursos_ce['TP_REDE'] = df_cursos_ce['TP_REDE'].replace(tp_rede)


### 1.4 Dataset total <a name="dataset-total"></a>

In [29]:
df_total = df_inst_ce.merge(df_cursos_ce, left_on=['NU_ANO_CENSO', 'CO_MUNICIPIO_IES', 'NO_MUNICIPIO_IES', 'CO_IES'], right_on=['NU_ANO_CENSO', 'CO_MUNICIPIO', 'NO_MUNICIPIO', 'CO_IES'], how='outer')

In [30]:
df_total.to_csv(graficos+'df_total_ies_cursos_ce.csv', index=False)

In [31]:
df_total.columns

Index(['NU_ANO_CENSO', 'SG_UF_IES', 'CO_MUNICIPIO_IES', 'NO_MUNICIPIO_IES',
       'IN_CAPITAL_IES', 'NO_MICRORREGIAO_IES', 'TP_ORGANIZACAO_ACADEMICA',
       'TP_CATEGORIA_ADMINISTRATIVA', 'CO_IES', 'NO_IES', 'SG_IES',
       'NO_MUNICIPIO', 'CO_MUNICIPIO', 'IN_CAPITAL', 'TP_REDE', 'NO_CURSO',
       'NO_CINE_ROTULO', 'NO_CINE_AREA_GERAL', 'TP_GRAU_ACADEMICO', 'QT_CURSO',
       'QT_ING', 'QT_ING_FINANC', 'QT_ING_FINANC_REEMB', 'QT_ING_FIES',
       'QT_ING_RPFIES', 'QT_ING_FINANC_REEMB_OUTROS', 'QT_ING_FINANC_NREEMB',
       'QT_ING_PROUNII', 'QT_ING_PROUNIP', 'QT_ING_NRPFIES',
       'QT_ING_FINANC_NREEMB_OUTROS', 'QT_MAT', 'QT_MAT_0_17', 'QT_MAT_18_24',
       'QT_MAT_25_29', 'QT_MAT_30_34', 'QT_MAT_35_39', 'QT_MAT_40_49',
       'QT_MAT_50_59', 'QT_MAT_60_MAIS', 'QT_MAT_FINANC',
       'QT_MAT_FINANC_REEMB', 'QT_MAT_FIES', 'QT_MAT_RPFIES',
       'QT_MAT_FINANC_REEMB_OUTROS', 'QT_MAT_FINANC_NREEMB', 'QT_MAT_PROUNII',
       'QT_MAT_PROUNIP', 'QT_MAT_NRPFIES', 'QT_MAT_FINANC_NREEMB

## 2. Análises e recortes para visualizações
### Episódio sobre financiamento estudantil

### 2.1 Proporção e números absolutos de matrículas com financiamento em relação ao total de alunos matriculados na rede privada de ensino superior no Ceará <a name="proporcao-e-absolutos"></a>

In [32]:
mat_ce = df_cursos_ce.loc[df_cursos_ce.TP_REDE == 'Privada'].groupby('NU_ANO_CENSO')['QT_MAT', 'QT_MAT_FINANC'].sum().astype(int).reset_index()

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\3254331127.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mat_ce = df_cursos_ce.loc[df_cursos_ce.TP_REDE == 'Privada'].groupby('NU_ANO_CENSO')['QT_MAT', 'QT_MAT_FINANC'].sum().astype(int).reset_index()


In [33]:
mat_ce['prop'] = round(mat_ce['QT_MAT_FINANC']/mat_ce['QT_MAT']*100,2)

In [34]:
mat_ce['QT_MAT_SEM_FINANC'] = mat_ce['QT_MAT']-mat_ce['QT_MAT_FINANC']

In [35]:
mat_ce.to_csv(graficos+'prop_matriculas_com_financiamento.csv', index=False)

In [36]:
mat_ce.columns = ['Ano','Total de Matriculas','Matrículas com financiamento','Proporção','Matriculas sem financiamento']
mat_ce

,Ano,Total de Matriculas,Matrículas com financiamento,Proporção,Matriculas sem financiamento
0,2011,114605,35211,30.72,79394
1,2012,133041,49825,37.45,83216
2,2014,175907,88465,50.29,87442
3,2015,189139,98988,52.34,90151
4,2016,198460,102447,51.62,96013
5,2017,192503,104973,54.53,87530
6,2018,201481,111677,55.43,89804
7,2019,220970,124377,56.29,96593
8,2020,233681,133329,57.06,100352
9,2021,235057,115553,49.16,119504


### 2.2 Números específicos de ingressantes e concluintes com financiamento estudantil <a name="ingressantes-e-concluintes"></a>

In [37]:
df_cursos_ce.groupby('NU_ANO_CENSO')['QT_ING_FINANC', 'QT_CONC_FINANC','QT_ING'].sum().astype(int).to_csv(graficos+'evolucao_anual_ingressantes_e_concluints_com_financiamento_e_total_ingressantes.csv')

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\2324082918.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_cursos_ce.groupby('NU_ANO_CENSO')['QT_ING_FINANC', 'QT_CONC_FINANC','QT_ING'].sum().astype(int).to_csv(graficos+'evolucao_anual_ingressantes_e_concluints_com_financiamento_e_total_ingressantes.csv')


In [38]:
df_cursos_ce.groupby('NU_ANO_CENSO')['QT_ING_FINANC', 'QT_CONC_FINANC','QT_ING'].sum().astype(int)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\774948144.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_cursos_ce.groupby('NU_ANO_CENSO')['QT_ING_FINANC', 'QT_CONC_FINANC','QT_ING'].sum().astype(int)


,QT_ING_FINANC,QT_CONC_FINANC,QT_ING
NU_ANO_CENSO,,,
2011,14508,2736,59292
2012,22911,3522,73742
2013,27848,4289,77954
2014,41551,5275,99100
2015,38381,7820,94040
2016,36917,10360,92204
2017,33261,12946,94542
2018,43207,16721,109102
2019,53565,17104,122045


In [66]:
# Total de alunos ingressantes e concluintes
df_cursos_ce.groupby('NU_ANO_CENSO')['QT_ING', 'QT_CONC'].sum().astype(int).to_csv(graficos+'total_alunos_ingressantes_e_concluintes.csv')

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\3502366783.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_cursos_ce.groupby('NU_ANO_CENSO')['QT_ING', 'QT_CONC'].sum().astype(int).to_csv(graficos+'total_alunos_ingressantes_e_concluintes.csv')


### 2.3 Proporção e números absolutos de matrículas na rede privada com financiamento estudantil em Fortaleza e no Interior do estado <a name="matriculas-com-financiamento"></a>

In [40]:
cap_int = df_cursos_ce.loc[df_cursos_ce.TP_REDE == 'Privada'].groupby([ 'IN_CAPITAL','NU_ANO_CENSO'])['QT_MAT_FINANC'].sum().reset_index().pivot(index='NU_ANO_CENSO', columns = 'IN_CAPITAL',values = 'QT_MAT_FINANC').reset_index()

In [41]:
cap_int['Total'] = cap_int['Interior']+cap_int['Capital']

In [42]:
cap_int['prop_int'] = round(cap_int['Interior']/cap_int['Total']*100,2)

In [43]:
cap_int.astype(int).to_csv(graficos+'matriculas_com_financiamento_capital_interior.csv', index=False)

In [44]:
cap_int 

IN_CAPITAL,NU_ANO_CENSO,Capital,Interior,Total,prop_int
0,2011,27510.0,7701.0,35211.0,21.87
1,2012,38729.0,11096.0,49825.0,22.27
2,2014,65768.0,22697.0,88465.0,25.66
3,2015,72505.0,26483.0,98988.0,26.75
4,2016,72732.0,29715.0,102447.0,29.01
5,2017,73541.0,31432.0,104973.0,29.94
6,2018,71069.0,40608.0,111677.0,36.36
7,2019,71017.0,53360.0,124377.0,42.90
8,2020,67927.0,65402.0,133329.0,49.05
9,2021,50128.0,65425.0,115553.0,56.62


### 2.4 Total de alunos com e sem financiamento estudantil da rede privada de ensino superior do Ceará <a name="total-com-e-sem-financiamento"></a>

In [45]:
total_de_matriculas_cap_int =  df_cursos_ce.loc[df_cursos_ce.TP_REDE == 'Privada'].groupby([ 'IN_CAPITAL','NU_ANO_CENSO'])['QT_MAT'].sum().reset_index().pivot(index='NU_ANO_CENSO', columns = 'IN_CAPITAL',values = 'QT_MAT').astype(int)

total_de_matriculas_cap_int.to_csv(graficos+'total_de_matriculas_cap_int.csv')

In [46]:
total_de_matriculas_cap_int

IN_CAPITAL,Capital,Interior
NU_ANO_CENSO,,
2011,85461,29144
2012,97511,35530
2014,131197,44710
2015,141270,47869
2016,147777,50683
2017,139580,52923
2018,136897,64584
2019,132719,88251
2020,132827,100854


### 2.5 Grandes áreas e tipos de grau acadêmico o financiamento estudantil no Ceará <a name="grandes-areas"></a>

In [47]:
df_area_cursos_ce = df_cursos_ce[['NU_ANO_CENSO', 'NO_MUNICIPIO', 'CO_MUNICIPIO', 'CO_IES', 'TP_REDE', 'NO_CURSO', 'NO_CINE_AREA_GERAL','QT_MAT','QT_MAT_FINANC']]
df_area_cursos_ce = arquivo_mapa_ceara[['CO_MUNICIPIO', 'Municipio', 'regiao']].merge(df_area_cursos_ce, on='CO_MUNICIPIO', how='right')

In [48]:
regiao_area_bolsas = df_area_cursos_ce.groupby(['regiao',	'NO_CINE_AREA_GERAL','NU_ANO_CENSO'])['QT_MAT_FINANC'].sum().reset_index().pivot(index=['regiao','NO_CINE_AREA_GERAL'], columns = 'NU_ANO_CENSO').fillna(0).astype(int).reset_index()

In [49]:
regiao_area_bolsas.columns = ['regiao','area','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']

In [50]:
regiao_area_bolsas.to_csv(graficos+'regiao_area_bolsas.csv', index=False)
regiao_area_bolsas

,regiao,area,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Cariri,"Agricultura, silvicultura, pesca e veterinária",0,0,0,0,0,0,59,253,474,405,488
1,Cariri,Artes e humanidades,1,0,0,0,0,0,1,2,91,82,103
2,Cariri,"Ciências naturais, matemática e estatística",0,0,0,0,0,0,0,0,2,9,9
3,Cariri,"Ciências sociais, comunicação e informação",464,513,623,738,748,810,737,859,910,955,884
4,Cariri,Computação e Tecnologias da Informação e Comun...,258,300,379,571,513,531,498,482,453,359,313
...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Vale do Jaguaribe,Educação,0,5,9,3,54,61,78,174,229,722,646
138,Vale do Jaguaribe,"Engenharia, produção e construção",0,0,0,0,9,7,6,29,47,104,86
139,Vale do Jaguaribe,"Negócios, administração e direito",21,38,51,13,137,201,270,548,604,1386,897
140,Vale do Jaguaribe,Saúde e bem-estar,14,30,21,12,61,82,79,122,99,698,784


In [51]:
financiamento_por_area_curso = df_cursos_ce.groupby([ 'NO_CINE_AREA_GERAL','NU_ANO_CENSO'])['QT_MAT_FINANC', 'QT_MAT'].sum().reset_index()

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\988975626.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  financiamento_por_area_curso = df_cursos_ce.groupby([ 'NO_CINE_AREA_GERAL','NU_ANO_CENSO'])['QT_MAT_FINANC', 'QT_MAT'].sum().reset_index()


In [52]:
financiamento_por_area_curso.pivot(index='NU_ANO_CENSO', columns = 'NO_CINE_AREA_GERAL',values = 'QT_MAT_FINANC')

NO_CINE_AREA_GERAL,"Agricultura, silvicultura, pesca e veterinária",Artes e humanidades,"Ciências naturais, matemática e estatística","Ciências sociais, comunicação e informação",Computação e Tecnologias da Informação e Comunicação (TIC),Educação,"Engenharia, produção e construção","Negócios, administração e direito",Programas básicos,Saúde e bem-estar,Serviços
NU_ANO_CENSO,,,,,,,,,,,
2011,40.0,1194.0,0.0,2316.0,2519.0,1164.0,1829.0,14978.0,0.0,10751.0,420.0
2012,138.0,1629.0,0.0,3550.0,3363.0,1801.0,3528.0,19694.0,0.0,15486.0,690.0
2013,318.0,1865.0,0.0,5107.0,3963.0,2359.0,6105.0,24507.0,0.0,21492.0,847.0
2014,521.0,2045.0,2.0,6544.0,5038.0,3785.0,9166.0,32523.0,0.0,27708.0,1133.0
2015,605.0,2182.0,2.0,7100.0,5198.0,4247.0,10859.0,37340.0,0.0,29876.0,1579.0
2016,731.0,1825.0,1.0,7139.0,5069.0,4708.0,11995.0,38290.0,0.0,31096.0,1593.0
2017,885.0,1436.0,2.0,8035.0,4647.0,5154.0,11948.0,39421.0,0.0,31883.0,1562.0
2018,1041.0,1250.0,6.0,7956.0,4805.0,8186.0,11901.0,40575.0,0.0,33313.0,2644.0
2019,1457.0,1546.0,51.0,7863.0,5138.0,13893.0,11938.0,43741.0,0.0,34247.0,4503.0


In [53]:
financiamento_por_area_curso.pivot(index='NU_ANO_CENSO', columns = 'NO_CINE_AREA_GERAL',values = 'QT_MAT_FINANC').to_csv(graficos+'financiamento_por_area.csv')

In [54]:
financiamento_por_area_curso = financiamento_por_area_curso.pivot(index='NO_CINE_AREA_GERAL', columns = 'NU_ANO_CENSO',values = 'QT_MAT_FINANC').reset_index()

In [55]:
financiamento_por_area_curso['var_20_21'] = round((financiamento_por_area_curso['2021']/financiamento_por_area_curso['2020']-1)*100,2)

In [56]:
financiamento_por_area_curso

NU_ANO_CENSO,NO_CINE_AREA_GERAL,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,var_20_21
0,"Agricultura, silvicultura, pesca e veterinária",40.0,138.0,318.0,521.0,605.0,731.0,885.0,1041.0,1457.0,1329.0,1610.0,21.14
1,Artes e humanidades,1194.0,1629.0,1865.0,2045.0,2182.0,1825.0,1436.0,1250.0,1546.0,1875.0,1692.0,-9.76
2,"Ciências naturais, matemática e estatística",0.0,0.0,0.0,2.0,2.0,1.0,2.0,6.0,51.0,149.0,132.0,-11.41
3,"Ciências sociais, comunicação e informação",2316.0,3550.0,5107.0,6544.0,7100.0,7139.0,8035.0,7956.0,7863.0,7072.0,6135.0,-13.25
4,Computação e Tecnologias da Informação e Comun...,2519.0,3363.0,3963.0,5038.0,5198.0,5069.0,4647.0,4805.0,5138.0,6291.0,5694.0,-9.49
5,Educação,1164.0,1801.0,2359.0,3785.0,4247.0,4708.0,5154.0,8186.0,13893.0,16865.0,14730.0,-12.66
6,"Engenharia, produção e construção",1829.0,3528.0,6105.0,9166.0,10859.0,11995.0,11948.0,11901.0,11938.0,10173.0,7322.0,-28.03
7,"Negócios, administração e direito",14978.0,19694.0,24507.0,32523.0,37340.0,38290.0,39421.0,40575.0,43741.0,47606.0,39316.0,-17.41
8,Programas básicos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,inf
9,Saúde e bem-estar,10751.0,15486.0,21492.0,27708.0,29876.0,31096.0,31883.0,33313.0,34247.0,34844.0,32684.0,-6.20


In [57]:
financ_area_ano = df_cursos_ce.loc[df_cursos_ce.TP_REDE == 'Privada'].groupby([ 'NO_CINE_AREA_GERAL','NU_ANO_CENSO'])['QT_MAT','QT_MAT_FINANC'].sum().astype(int).reset_index()

financ_area_ano['prop'] = round(financ_area_ano['QT_MAT_FINANC']/financ_area_ano['QT_MAT']*100,2)

financ_area_ano

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\658151069.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  financ_area_ano = df_cursos_ce.loc[df_cursos_ce.TP_REDE == 'Privada'].groupby([ 'NO_CINE_AREA_GERAL','NU_ANO_CENSO'])['QT_MAT','QT_MAT_FINANC'].sum().astype(int).reset_index()


,NO_CINE_AREA_GERAL,NU_ANO_CENSO,QT_MAT,QT_MAT_FINANC,prop
0,"Agricultura, silvicultura, pesca e veterinária",2011,577,40,6.93
1,"Agricultura, silvicultura, pesca e veterinária",2012,710,138,19.44
2,"Agricultura, silvicultura, pesca e veterinária",2014,1064,521,48.97
3,"Agricultura, silvicultura, pesca e veterinária",2015,1282,605,47.19
4,"Agricultura, silvicultura, pesca e veterinária",2016,1362,731,53.67
...,...,...,...,...,...
94,Serviços,2017,3448,1562,45.30
95,Serviços,2018,5165,2644,51.19
96,Serviços,2019,7185,4503,62.67
97,Serviços,2020,10010,7125,71.18


In [58]:
financ_area_ano.sort_values(by=['prop'], ascending=False)

,NO_CINE_AREA_GERAL,NU_ANO_CENSO,QT_MAT,QT_MAT_FINANC,prop
21,"Ciências naturais, matemática e estatística",2015,2,2,100.00
22,"Ciências naturais, matemática e estatística",2016,1,1,100.00
23,"Ciências naturais, matemática e estatística",2017,2,2,100.00
97,Serviços,2020,10010,7125,71.18
26,"Ciências naturais, matemática e estatística",2020,222,149,67.12
...,...,...,...,...,...
1,"Agricultura, silvicultura, pesca e veterinária",2012,710,138,19.44
48,Educação,2011,6234,1164,18.67
24,"Ciências naturais, matemática e estatística",2018,33,6,18.18
78,Programas básicos,2021,68,9,13.24


In [59]:
financ_area_ano = df_cursos_ce.loc[df_cursos_ce.TP_REDE == 'Privada'].groupby([ 'NO_CINE_AREA_GERAL'])['QT_MAT','QT_MAT_FINANC'].sum().astype(int).reset_index()

financ_area_ano['prop'] = round(financ_area_ano['QT_MAT_FINANC']/financ_area_ano['QT_MAT']*100,2)

financ_area_ano.sort_values(by=['prop'], ascending=False)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\739855433.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  financ_area_ano = df_cursos_ce.loc[df_cursos_ce.TP_REDE == 'Privada'].groupby([ 'NO_CINE_AREA_GERAL'])['QT_MAT','QT_MAT_FINANC'].sum().astype(int).reset_index()


,NO_CINE_AREA_GERAL,QT_MAT,QT_MAT_FINANC,prop
3,"Ciências sociais, comunicação e informação",110373,63710,57.72
9,Saúde e bem-estar,510109,281888,55.26
6,"Engenharia, produção e construção",165561,90659,54.76
4,Computação e Tecnologias da Informação e Comun...,88600,47755,53.90
10,Serviços,51141,27462,53.70
2,"Ciências naturais, matemática e estatística",658,345,52.43
1,Artes e humanidades,34950,16672,47.70
7,"Negócios, administração e direito",748841,353460,47.20
0,"Agricultura, silvicultura, pesca e veterinária",18019,8357,46.38
5,Educação,166524,74528,44.76


### 2.6 Outras consultas (Extra) <a name="extra"></a>


In [60]:
df_cursos_ce.groupby('NU_ANO_CENSO')['QT_VG_TOTAL'].sum().astype(int).reset_index()

,NU_ANO_CENSO,QT_VG_TOTAL
0,2011,65587
1,2012,72316
2,2013,104319
3,2014,148656
4,2015,182005
5,2016,214787
6,2017,212759
7,2018,215841
8,2019,227781
9,2020,230038


In [61]:
qtd_vagas = df_cursos_ce.groupby(['NU_ANO_CENSO','TP_REDE'])['QT_VG_TOTAL'].sum().astype(int).reset_index()
qtd_vagas.pivot(index='NU_ANO_CENSO',columns='TP_REDE').reset_index()

NU_ANO_CENSO QT_VG_TOTAL        
TP_REDE                  Privada Pública
0               2011       48295   17292
1               2012       55154   17162
2               2014      125803   22853
3               2015      156258   25747
4               2016      189002   25785
5               2017      186780   25979
6               2018      187415   28426
7               2019      199822   27959
8               2020      203415   26623
9               2021      228508   22758

In [62]:
qtd_vagas = df_cursos_ce.groupby(['NU_ANO_CENSO','NO_CINE_AREA_GERAL'])['QT_VG_TOTAL'].sum().astype(int).reset_index()
qtd_vagas.pivot(index='NU_ANO_CENSO',columns='NO_CINE_AREA_GERAL').reset_index()

NU_ANO_CENSO  \
NO_CINE_AREA_GERAL                
0                          2011   
1                          2012   
2                          2013   
3                          2014   
4                          2015   
5                          2016   
6                          2017   
7                          2018   
8                          2019   
9                          2020   
10                         2021   

                                                      QT_VG_TOTAL  \
NO_CINE_AREA_GERAL Agricultura, silvicultura, pesca e veterinária   
0                                                            1054   
1                                                            1126   
2                                                            1276   
3                                                            1326   
4                                                            1577   
5                                                            1775   
6                                                            2335   
7                                                            3017   
8                                                            2898   
9                                                            3253   
10                                                           3627   

                                        \
NO_CINE_AREA_GERAL Artes e humanidades   
0                                 2649   
1                                 3314   
2                                 3456   
3                                 4486   
4                                 7043   
5                                 8429   
6                                 7254   
7                                 6986   
8                                 7193   
9                                 5377   
10                                5265   

                                                                \
NO_CINE_AREA_GERAL Ciências naturais, matemática e estatística   
0                                                          785   
1                                                          737   
2                                                          808   
3                                                          820   
4                                                          802   
5                                                          815   
6                                                          888   
7                                                         1170   
8                                                         1095   
9                                                         1528   
10                                                        1279   

                                                               \
NO_CINE_AREA_GERAL Ciências sociais, comunicação e informação   
0                                                        3616   
1                                                        3917   
2                                                        6760   
3                                                        8349   
4                                                       10960   
5                                                       14226   
6                                                       11394   
7                                                       13565   
8                                                       12001   
9                                                       17356   
10                                                      14312   

                                                                               \
NO_CINE_AREA_GERAL Computação e Tecnologias da Informação e Comunicação (TIC)   
0                                                                5433           
1                                                                6092           
2                                                                9330           
3       

In [63]:


qtd_vagas = df_cursos_ce.loc[df_cursos_ce.TP_REDE == 'Privada'].groupby(['NU_ANO_CENSO','NO_CINE_AREA_GERAL'])['QT_VG_TOTAL'].sum().astype(int).reset_index()
qtd_vagas.pivot(index='NU_ANO_CENSO',columns='NO_CINE_AREA_GERAL').reset_index().fillna(0).astype(int)

NU_ANO_CENSO  \
NO_CINE_AREA_GERAL                
0                          2011   
1                          2012   
2                          2014   
3                          2015   
4                          2016   
5                          2017   
6                          2018   
7                          2019   
8                          2020   
9                          2021   

                                                      QT_VG_TOTAL  \
NO_CINE_AREA_GERAL Agricultura, silvicultura, pesca e veterinária   
0                                                             285   
1                                                             305   
2                                                             403   
3                                                             559   
4                                                             593   
5                                                            1133   
6                                                            1603   
7                                                            1695   
8                                                            1980   
9                                                            2582   

                                        \
NO_CINE_AREA_GERAL Artes e humanidades   
0                                 2190   
1                                 2470   
2                                 3491   
3                                 6017   
4                                 7340   
5                                 6181   
6                                 5793   
7                                 6301   
8                                 4540   
9                                 4584   

                                                                \
NO_CINE_AREA_GERAL Ciências naturais, matemática e estatística   
0                                                            0   
1                                                            0   
2                                                            0   
3                                                            0   
4                                                            0   
5                                                            0   
6                                                          244   
7                                                          115   
8                                                          575   
9                                                          583   

                                                               \
NO_CINE_AREA_GERAL Ciências sociais, comunicação e informação   
0                                                        2543   
1                                                        2808   
2                                                        7081   
3                                                        9456   
4                                                       12755   
5                                                        9953   
6                                                       11967   
7                                                       10445   
8                                                       15833   
9                                                       13022   

                                                                               \
NO_CINE_AREA_GERAL Computação e Tecnologias da Informação e Comunicação (TIC)   
0                                                                4760           
1                                                                5410           
2                                                               10879           
3                                                               11638           
4                                                               14592           
5                                                               14746           
6                                         

In [64]:
df_cursos_ce.groupby('NU_ANO_CENSO')['QT_MAT_PROUNII','QT_MAT_PROUNIP','QT_MAT_FIES','QT_MAT_RPFIES','QT_MAT_NRPFIES'].sum().astype(int)

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_2084\3171230181.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_cursos_ce.groupby('NU_ANO_CENSO')['QT_MAT_PROUNII','QT_MAT_PROUNIP','QT_MAT_FIES','QT_MAT_RPFIES','QT_MAT_NRPFIES'].sum().astype(int)


,QT_MAT_PROUNII,QT_MAT_PROUNIP,QT_MAT_FIES,QT_MAT_RPFIES,QT_MAT_NRPFIES
NU_ANO_CENSO,,,,,
2011,4010,1448,14713,106,16978
2012,4918,2291,25251,736,20968
2013,5304,2287,39561,216,26664
2014,7012,2289,63127,2,34181
2015,9216,2852,65949,92,39308
2016,11395,4015,63563,621,30197
2017,12578,3642,58725,4003,44185
2018,11291,4642,48980,3616,55317
2019,13429,6324,42218,5997,69190
